# Čtvrté cvičení - vyhlazování jazykového modelu
- Vytvořte bigramový jazykový model vyhlazený metodou Witten-Bell pro soubor TEXTEN1
- Soubor přeuspořádejte tak, aby každá věta byla na jednom řádku (jako oddělovač vět použijte tečku, otazník a vykřičník).
- Všechny slova převeďte na lowercase
- Uvažujte na začátku věty startovací symbol <s>

In [1]:
import re, string
...
...
...
print(en_bigram_model["of the"])
#0.258387066629287
print(en_bigram_model["of of"])
#1.6608937523330386e-05